In [1]:
import pandas as pd
import numpy as np
import surprise
import sklearn
from surprise import KNNWithMeans,accuracy,Reader,Dataset
from surprise.model_selection import train_test_split

In [26]:
#loading all csv files
ratings = pd.read_csv("ratings.csv")
movies_data = pd.read_csv('movies.csv')
links = pd.read_csv('links.csv')
tags = pd.read_csv('tags.csv')
movies_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


####    1.Create recommender system using ratings.csv file. 

In [3]:
data_pvt=ratings.pivot(index='userId',columns='movieId',values='rating')
data_pvt.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data_pvt.shape

(610, 9724)

In [5]:
#reading csv file data using reader
reader = Reader(rating_scale=(1,5)) # movie rating sacle is 1 to 5
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']],reader) # reader takes u_id,m_id,rating as values

In [6]:
[trainset,testset] = train_test_split(data,test_size=0.15,shuffle=True)

recom_sys = KNNWithMeans(k=65, sim_options={"Name":"Cosine","user_based":True}) # using cosine semilarity

recom_sys.fit(data.build_full_trainset())

test_pred = recom_sys.test(testset)

RMSE = accuracy.rmse(test_pred)

print("RMSE =",RMSE)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.7171
RMSE = 0.7171287680697583


#### 2.Display predicted rating for a particular userID and MovieID combinations (both taken as keyboard input)

In [7]:
user_id=int(input('Enter the user id : ')) #taking the user_id as keybord input
mvi_id = int(input('Enter the movie id :'))#taking the mvi_id as keybord input
print("rating for movie_id",mvi_id,"by user_id",user_id,"is:",round(recom_sys.predict(user_id,mvi_id)[3],2))
print("\n")

Enter the user id : 55
Enter the movie id :66
rating for movie_id 66 by user_id 55 is: 1.68




#### 3.Recommend a user to watch top 10 movies, which has not watched yet (as per his rating predictions). Take userID as a keyboard input. Fix the threshold rating as 2.5

In [8]:
data_pvt.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
recom_sys.predict(66,88)[3]

3.706252078704763

In [10]:
usr_mvi={}
U_id = int(input('Enter user id: '))
threshold = float(input('Enter the threshold value: '))
for m_id in data_pvt.columns:
    if recom_sys.predict(U_id,m_id)[3] > threshold:
        usr_mvi[m_id]=recom_sys.predict(U_id,m_id)[3]        

Enter user id: 55
Enter the threshold value: 2.5


In [11]:
usr_mvi

{1: 2.8917526012201162,
 2: 2.6799748374403904,
 3: 2.5367959259373603,
 6: 3.244904560370091,
 10: 2.9059265121995956,
 11: 2.6793884897656097,
 13: 2.6772302105163943,
 14: 2.8366251902210027,
 16: 3.3012015658706977,
 17: 3.1502854945123344,
 18: 3.151957471270295,
 21: 2.858957242928353,
 22: 2.686493691626864,
 25: 2.9855592065366325,
 26: 2.8789061521696153,
 27: 2.527954026684085,
 28: 3.2179979623637243,
 29: 3.3232202075900843,
 30: 3.3091066286744057,
 32: 3.1196640500134345,
 34: 2.5987113908938326,
 36: 3.2027091214082084,
 39: 2.5672388041048135,
 40: 2.84,
 41: 3.412938027351405,
 45: 2.602647379128323,
 47: 3.2877776533772254,
 50: 3.339167945126253,
 52: 2.982532430716425,
 53: 4.288891943891828,
 55: 3.150580204778157,
 58: 3.203297346851973,
 60: 2.6870988742027553,
 61: 3.1972769374483487,
 62: 2.7656643361510436,
 69: 2.87398737777307,
 70: 2.8774668776045234,
 72: 2.5367425669563355,
 73: 2.8211174240652297,
 74: 2.619023212018089,
 77: 3.3320466595970304,
 80: 3.2

In [27]:
from collections import Counter
n=Counter(usr_mvi)


popular = n.most_common(10)# using most_common method to find the most commonly view movies 
print('movies recommended for user',U_id,'are:\n')
for i in range(10):
    print(i+1,'.',list(movies_data[movies_data.movieId==popular[i][0]]['title'])[0])

movies recommended for user 55 are:

1 . Hangar 18 (1980)
2 . Galaxy of Terror (Quest) (1981)
3 . Alien Contamination (1980)
4 . Jetée, La (1962)
5 . Match Factory Girl, The (Tulitikkutehtaan tyttö) (1990)
6 . Paterson
7 . The Big Bus (1976)
8 . Seve (2014)
9 . Android (1982)
10 . Looker (1981)


In [28]:
#list(movies_data[movies_data.movieId==popular[9][0]]['title'])[0]

#### 4.Display the MovieID, IMDB ID, Average User Rating (excluding predictions), genres and tag of all the movies found in Step 3 as a data frame.

In [29]:
imdb_id=list(links[links.movieId==popular[9][0]]['imdbId'])

In [30]:
tag=list(tags[tags.movieId==popular[2][0]]['tag'])
tag

[]

In [31]:
#calculating average user rating 
for m_id in data_pvt.columns:
    ratings['avg_rating']= round(ratings[ratings['movieId'] == m_id]['rating'].mean(),2)

In [32]:
#ratings[ratings['movieId'] == m_id]['rating'].mean()

In [33]:
ratings=ratings.drop(['timestamp'],axis=1)
ratings.head()

,userId,movieId,rating,avg_rating
0,1,1,4.0,4.0
1,1,3,4.0,4.0
2,1,6,4.0,4.0
3,1,47,5.0,4.0
4,1,50,5.0,4.0


In [34]:
movies_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [35]:
#merging all the required columns together into a dataframe
main_df=pd.merge(links[['movieId','imdbId']],ratings[['movieId','avg_rating']],how='left',on='movieId')
main_df=pd.merge(main_df,movies_data[['movieId','genres']],how='left',on='movieId')
main_df=pd.merge(main_df,tags[['movieId','tag']],how='left',on='movieId')
main_df.head()

,movieId,imdbId,avg_rating,genres,tag
0,1,114709,4.0,Adventure|Animation|Children|Comedy|Fantasy,pixar
1,1,114709,4.0,Adventure|Animation|Children|Comedy|Fantasy,pixar
2,1,114709,4.0,Adventure|Animation|Children|Comedy|Fantasy,fun
3,1,114709,4.0,Adventure|Animation|Children|Comedy|Fantasy,pixar
4,1,114709,4.0,Adventure|Animation|Children|Comedy|Fantasy,pixar


In [36]:
ls = []    
for i in range(10):
    ls.append(popular[i][0])
final = main_df[main_df["movieId"].isin(ls)]
final

,movieId,imdbId,avg_rating,genres,tag
211487,5181,80836,4.0,Action|Sci-Fi|Thriller,NaN
211488,5181,80836,4.0,Action|Sci-Fi|Thriller,NaN
213588,5490,74205,4.0,Action|Comedy,NaN
214994,5746,82431,4.0,Action|Horror|Mystery|Sci-Fi,NaN
215006,5764,82677,4.0,Drama|Horror|Sci-Fi|Thriller,NaN
215612,5919,83557,4.0,Sci-Fi,NaN
215613,5919,83557,4.0,Sci-Fi,NaN
222096,6835,82000,4.0,Action|Horror|Sci-Fi,NaN
233055,8477,56119,4.0,Romance|Sci-Fi,post-apocalyptic
233056,8477,56119,4.0,Romance|Sci-Fi,post-apocalyptic


In [37]:
final['movieId'].unique()

array([  5181,   5490,   5746,   5764,   5919,   6835,   8477,  40491,
       132333, 156605], dtype=int64)

* there are multiple tags for a single movie 